# With only males and females individuals (which fits the capture)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import math

In [ ]:
def temperature(t, offset = 0): # Temperature function in kelvin
    return (4*np.cos(2*np.pi*(t + offset)/365.25 + 9.2) + 28.0) + 273.15
    
def temperature_const(t,offset = 0): # Constant temperature
    return 273.15 + 28.0

def humidity(t, offset = 0): # Humidity function in percent
    return 60 #+ 30*np.cos(6*np.pi*(t + offset)/365.25 + 9.2) + 10*np.cos(8*np.pi*(t + offset)/365.25 + 9.2)

def transition_rate(evt, T, R0, Ha, Hh, T12, R): # Transition rate function from one state to another
    return R0[evt]*(T/298)*np.exp(Ha[evt]*(1/298 - 1/T)/R)/(1 + np.exp(Hh[evt]*(1/T12[evt] - 1/T)/R))

def adult_rate(T, R0, Ha, Hh, T12, R): # Transition rate function from larvae to pupae to adult
    egg_transi = transition_rate(0, T, R0, Ha, Hh, T12, R)
    larvae_transi = transition_rate(1, T, R0, Ha, Hh, T12, R)
    pupae_transi = transition_rate(2, T, R0, Ha, Hh, T12, R)
    death_L = 0.01 + 0.9725*np.exp(-0.1*(T - 278)/2.7035)
    death_P = 0.01 + 0.9725*np.exp(-0.1*(T - 278)/2.7035)

    return egg_transi*larvae_transi*pupae_transi/((pupae_transi + death_P) * (larvae_transi + death_L))

# definition of the different mortality rates : 
def egg_death(T) : 
    if T < 288 : 
        return 0
    else :
        return 0.011 # A calculer
    
def racine_pos(a, b, c, d, F):
    rp = ((a + b) + math.sqrt((a + b)**2 + 4 * c * d * F)) / (d * F)
    if rp < 0:
        return 0
    return rp


In [ ]:
def mating(M, Ms):
    return M / (1 + M + Ms)

def event(i, state, M_event):
    return state + M_event[i]

def release1(M, n):
    return n*M + 2000

def competition(K0, Kh, precip):
    return 1/(K0 + Kh *  precip)

In [ ]:
#open the meteorological data
import pandas as pd
meteo_df = pd.read_csv('../Data/meteo_tetiaroa_resampled.csv')
print("Columns in meteo_df:", meteo_df.columns)

precip_data = meteo_df['precip2'].values
temperature_data = meteo_df['TM'].values
dates_data = meteo_df['date'].values
time_data = meteo_df['time'].values

In [ ]:
dates_data

# Population model

In [ ]:
def det_model(y, t, birth, n_egg, deltaA, deltaE, comp, mu, R0, Ha, Hh, T12, R, temperature_data, precip_data):
    F, M, Ms = y

    #T = temperature(t)
    #T = temperature_const(t)
    T = temperature_data[int(t*100001/1552)-1] + 273.15
    #precip = precip_data[int(t*100001/1552)-1]
    precip = 0


    if M + Ms == 0:
        probaM = 0
        probaMs = 0
    else:
        probaM = 2 * M / (2 * M + Ms)
        probaMs = Ms / (2 * M + Ms)
    
    transi = adult_rate(T, R0, Ha, Hh, T12, R)
    death_egg = egg_death(T) + competition(1/comp, 1/(comp * 30), precip) * (M + F) + (transition_rate(0, T, R0, Ha, Hh, T12, R) - transi)
    compet = competition(1/comp, 1/(comp * 30), precip)
    matf = mating(0.01*M, 0.001*Ms)* probaM
    deltaF = deltaA
    deltaM = deltaA * 2 

    Ep = racine_pos(deltaE + (transition_rate(0, T, R0, Ha, Hh, T12, R)), transi, compet, birth, F)

    # Differential equations
    dF_dt = transi * Ep / 2  - deltaF * F
    dM_dt = transi * Ep / 2 - deltaM * 2 * M
    dMs_dt = - deltaA * 1.2 * Ms
    return [dF_dt, dM_dt, dMs_dt]

def sim(n_intervals, pop_init, days, birth, deltaA, deltaE, comp, mu, R0, Ha, Hh, T12, R, n_egg=64, temperature_data=temperature_data, precip_data=precip_data):
    # Solve the differential equations
    pop0 = pop_init
    F, M, Ms = [], [], []
    # Divide the time in n_intervals
    taille_intervals = len(days)//n_intervals
    intervals = [days[i * taille_intervals : (i + 1) * taille_intervals] for i in range(n_intervals - 1)] + [days[(n_intervals - 1) * taille_intervals :]]

    for interval in intervals:
        solution = odeint(
            det_model, pop0, interval,
            args=(birth, n_egg, deltaA, deltaE, comp, mu, R0, Ha, Hh, T12, R, temperature_data, precip_data)
        )
        F0, M0, Ms0 = solution.T
        F.extend(F0)
        M.extend(M0)
        Ms.extend(Ms0)

        if interval[-1] > 750 and F[-1] > 200 and 3 ==4:
            Ms[-1] = Ms[-1] + release1(M[-1],50)
            #print("release", Ms[-1])

        pop0 = [F[-1], M[-1], Ms[-1]]

    return F, M, Ms

In [ ]:
# Ensure that days and release are defined
#days = np.linspace(0, 3600, 36001)  # Time steps
#release = np.arange(7, 3600.1, 7)  # Release every week starting from day 7

# Check if any element in days is close to any element in release
#result = np.any(np.abs(days[:, None] - release) < 1e-3)
#print(result)

In [ ]:

# Parameters
birth = 0.05  # Birth rate per adult
puberty = 0.83      # transition rate from egg to adult
mu = 0.5    # rate at which egg becomes a female
deltaE = 0.2     # Death rate per egg
deltaA = 0.091  # Death rate per adult
init_f = 100  # Starting female population
init_male = init_f  # Starting adult population
init_sterile = 0  # Starting sterile male population
max_time = 700  # Maximum simulation time
comp = 0.0001  # Carrying capacity
n_egg = 64

R0 = [0.24, 0.2088, 0.384]
Hh = [100, 55.99, -472.379]
Ha = [10.798, 26.018, 14.931]
T12 = [14.184, 304.6, 148]
R = 8.31446261815324 # Universal gas constant
# Initialize variables

n_intervals = 1552//7
days = np.linspace(0, 1552, 155100)  # Time steps
pop = np.array([init_f, init_male, init_sterile])  # Initial population

In [ ]:
# Plot temperature and precipitation in two separate graphs
fig, ax1 = plt.subplots(figsize=(10, 4))
# Convert dates_data to pandas datetime for better plotting
dates_dt = pd.to_datetime(dates_data)
ax1.plot(dates_dt, temperature_data, label='Temperature (°C)', color='tab:red')
ax1.set_xlabel('Days')
ax1.set_ylabel('Temperature (°C)', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.set_title('Temperature over Time')
plt.legend()
plt.tight_layout()
plt.show()

fig, ax2 = plt.subplots(figsize=(10, 4))
ax2.plot(dates_dt, precip_data, label='Precipitation (mm)', color='tab:blue')
ax2.set_xlabel('Days')
ax2.set_ylabel('Precipitation (mm)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_title('Precipitation over Time')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Run the simulation
F, M, Ms= sim(n_intervals, pop, days, birth, deltaA, deltaE, comp, mu, R0 , Ha, Hh , T12, R)
print(max(F), max(M), max(Ms))
print(min(F), min(M), min(Ms))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Plot female and male population
axs[0].step(dates_dt, F, where="post", label='Female', color='blue')
axs[0].step(dates_dt, M, where="post", label='Male', color='black')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Population Size')
axs[0].set_title('Female and Male Population')
axs[0].legend()

# Plot sterile male population
axs[1].step(dates_dt, np.array(F) - np.array(M), where="post", label='Sterile Male', color='orange')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Sterile Male Population Size')
axs[1].set_ylim(bottom=0)
axs[1].set_title('Sterile Male Population')
axs[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 7))
# Plot results for male and female population
axs[0].step(days, np.array(F) + np.array(Fs) + np.array(M) + np.array(Ff), where="post", label='Adult', color = 'blue')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Population Size')
axs[0].set_title('Adult Population')
axs[0].legend()

# Plot results for sterile male population
axs[1].step(days, Ms, where="post", label='Sterile Male', color='orange')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Sterile Male Population Size')
axs[1].set_ylim(bottom=0)
axs[1].set_title('Sterile Male Population')
axs[1].legend()